# Домашнее задание к лекции "Понятие класса"

### Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [1]:
# импортируем библиотеку requests, которая является стандартным инструментом для составления HTTP-запросов в Python
import requests

class Rate:
    
    # служебный метод, выполняется при вызове класса
    def __init__(self, format_='value'):  
        self.format = format_
    
    def exchange_rates(self):
        
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        
        # распаковка json-формата в строчку
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        - краткая информация о валюте при self.format == 'value':
        Rate('value').make_format('EUR')
        79.4966
        """
        
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        
        """Возвращает курс евро на сегодня в формате self.format"""
        
        return self.make_format('EUR')
    
    def usd(self):
        
        """Возвращает курс доллара на сегодня в формате self.format"""
        
        return self.make_format('USD')
    
    def brl(self):
        
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        
        return self.make_format('BRL')
    
    def bgn(self):
        
        """Возвращает курс болгарского льва на сегодня в формате self.format"""
        
        return self.make_format('BGN')
    
    def name_max_rates(self):
        
        """возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса
        https://www.cbr-xml-daily.ru/daily_json.js"""
        
        # создадим пустой словарь, куда будет записываться результат (название валюты(поле ‘Name’):значение курса валюты)
        result = {}
        
        # извлекаем данные из файла в переменную response
        response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
        
        # проходим циклом по словарю
        for currency in response:
            result[response[currency]['Name']] =  response[currency]['Value']/response[currency]['Nominal']
            
        # выделяем из словаря название валюты с максимальным значением курса
        max_value = max(result.items(), key=lambda x: x[1])   
        return max_value[0]

In [2]:
# вызываем класс Rate, смотрим полную информацию о валюте при self.format = 'full'
r = Rate(format_ = 'full')

In [3]:
r.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 78.1281,
 'Previous': 78.0915}

In [4]:
r.brl()

{'ID': 'R01115',
 'NumCode': '986',
 'CharCode': 'BRL',
 'Nominal': 1,
 'Name': 'Бразильский реал',
 'Value': 13.747,
 'Previous': 13.8364}

In [5]:
# вызываем класс Rate, смотрим краткую информацию о валюте при self.format = 'value'
r = Rate(format_ = 'value')

In [6]:
r.usd()

78.1281

In [7]:
r.brl()

13.747

In [8]:
r = Rate(format_ = 'full')
r.name_max_rates()

'СДР (специальные права заимствования)'

### Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [9]:
class Diff_rate(Rate):
    
    # служебный метод, выполняется при вызове класса, добавим параметр diff
    def __init__(self, format_, diff = True):
        
        # наследуем разработку Rate и полезные функции передадим Diff_rate 
        super().__init__()
        
        self.format = format_
        self.diff = diff

    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        или изменение курса по сравнению в прошлым значением
        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]

            elif self.format == 'value' and self.diff:
                return round(response[currency]['Value'] - response[currency]['Previous'], 2)

In [10]:
df = Diff_rate(format_='full')

In [11]:
df.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 78.1281,
 'Previous': 78.0915}

In [12]:
df= Diff_rate(format_='value')

In [13]:
df.usd()

0.04

### Задание 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла“). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

In [14]:
class Employee:
    
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        self.grade = 1
    
    def grade_up(self):
        
        """Повышает уровень сотрудника"""
        
        self.grade += 1
    
    def publish_grade(self):
        
        """Публикация результатов аккредитации сотрудников"""
        
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [15]:
class Designer(Employee):
    
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards

    def check_awards(self):
        self.seniority += 2 * self.awards

    def check_if_it_is_time_for_upgrade(self):
        
        # по условию увеличиваем аккредитации за имеющиеся международные премии при выходе на работу
        if self.seniority == 0:
            self.check_awards()
            
        # для каждой аккредитации увеличиваем счетчик на 1
        self.seniority += 1

        # условие повышения сотрудника
        if self.seniority % 7 == 0:
            self.grade_up()

        # публикация результатов
        return self.publish_grade()

In [16]:
elena = Designer('Елена', seniority=0, awards=2)
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4
